# Get monthly statisitcs from analysis ready cloud optimized (ARCO) ERA5 data
Follows the tutorials:
- https://github.com/google-research/arco-era5/blob/main/docs/0-Surface-Reanalysis-Walkthrough.ipynb
- https://github.com/google-research/arco-era5/blob/main/docs/1-Model-Levels-Walkthrough.ipynb

## List available data

In [1]:

import fsspec

fs = fsspec.filesystem('gs')
fs.ls('gs://gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2/')

['gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2/.zattrs',
 'gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2/.zgroup',
 'gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2/.zmetadata',
 'gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2/10m_u_component_of_wind',
 'gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2/10m_v_component_of_wind',
 'gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2/2m_temperature',
 'gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2/angle_of_sub_gridscale_orography',
 'gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2/anisotropy_of_sub_gridscale_orography',
 'gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2/geopotential',
 'gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2/geopotential_at_surface',
 'gcp-public-data-arco

## Get ARCO data

In [2]:
import xarray as xr

arco_era5 = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2', 
    chunks={'time': 48},
    consolidated=True,
)

In [3]:
print(f'size: {arco_era5.nbytes / (1024 ** 4)} TiB')

size: 486.0272591067878 TiB


In [4]:
arco_era5

<xarray.Dataset>
Dimensions:                                           (time: 552264,
                                                       latitude: 721,
                                                       longitude: 1440,
                                                       level: 37)
Coordinates:
  * latitude                                          (latitude) float32 90.0...
  * level                                             (level) int64 1 2 ... 1000
  * longitude                                         (longitude) float32 0.0...
  * time                                              (time) datetime64[ns] 1...
Data variables: (12/31)
    10m_u_component_of_wind                           (time, latitude, longitude) float32 dask.array<chunksize=(48, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                           (time, latitude, longitude) float32 dask.array<chunksize=(48, 721, 1440), meta=np.ndarray>
    2m_temperature                                    (time, latitude, longitude) float32 dask.array<chunksize=(48, 721, 1440), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    geopotential                                      (time, level, latitude, longitude) float32 dask.array<chunksize=(48, 37, 721, 1440), meta=np.ndarray>
    ...                                                ...
    total_precipitation                               (time, latitude, longitude) float32 dask.array<chunksize=(48, 721, 1440), meta=np.ndarray>
    type_of_high_vegetation                           (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    type_of_low_vegetation                            (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    u_component_of_wind                               (time, level, latitude, longitude) float32 dask.array<chunksize=(48, 37, 721, 1440), meta=np.ndarray>
    v_component_of_wind                               (time, level, latitude, longitude) float32 dask.array<chunksize=(48, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity                                 (time, level, latitude, longitude) float32 dask.array<chunksize=(48, 37, 721, 1440), meta=np.ndarray>

In [5]:
arco_era5.temperature.attrs

{'long_name': 'Temperature',
 'short_name': 't',
 'standard_name': 'air_temperature',
 'units': 'K'}

## Explore data

In [6]:
# select 1 year
arco_era5_2018 = arco_era5[['2m_temperature', 'total_precipitation', 'total_cloud_cover']].sel(time=slice('2018-01-01', '2019-01-01'))
arco_era5_2018_month = arco_era5_2018.resample(time="1M")

In [7]:
arco_era5_2018

<xarray.Dataset>
Dimensions:              (time: 8784, latitude: 721, longitude: 1440)
Coordinates:
  * latitude             (latitude) float32 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude            (longitude) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * time                 (time) datetime64[ns] 2018-01-01 ... 2019-01-01T23:0...
Data variables:
    2m_temperature       (time, latitude, longitude) float32 dask.array<chunksize=(48, 721, 1440), meta=np.ndarray>
    total_precipitation  (time, latitude, longitude) float32 dask.array<chunksize=(48, 721, 1440), meta=np.ndarray>
    total_cloud_cover    (time, latitude, longitude) float32 dask.array<chunksize=(48, 721, 1440), meta=np.ndarray>

In [8]:
%%time
#plot 2m temperature for 1 timestamp
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

subset = arco_era5_2018['2m_temperature'].sel(time='2018-01-01T12:00:00.000000000', longitude=slice(62,63),latitude=slice(69,67))#, level=1)

ax = plt.subplot(projection=ccrs.PlateCarree())

subset.plot.pcolormesh(x="longitude", y="latitude", ax=ax, infer_intervals=True)
ax.coastlines()
ax.gridlines(draw_labels=True)

CPU times: total: 0 ns
Wall time: 0 ns


In [9]:
# helper function
def lon_to_360(dlon: float) -> float:
  return ((360 + (dlon % 360)) % 360)

## Get monthly statistics for cities

In [10]:
%%time
import pandas as pd
city_geom  = "s3/data/d001_administration/urban_audit_city_2021/URAU_RG_100K_2021_4326_CITIES/city_working_lat_lon_list.csv"
city_all = pd.read_csv(city_geom)

# get city coordinates
# lonlat_list =[["NL005C", 4.640960, 52.113299], ["NL006C", 5.384670, 52.173656], ["NL007C", 5.921886, 52.189884]]
lon_list = [lon_to_360(val) for val in city_all["LON"].values.tolist()]
lat_list = city_all["LAT"].values.tolist()
city_list = city_all["URAU_CODE"].values.tolist()
target_lon = xr.DataArray(lon_list, dims="city", coords={"city": city_list})
target_lat = xr.DataArray(lat_list, dims="city", coords={"city": city_list})

CPU times: total: 0 ns
Wall time: 0 ns


In [27]:
years= range(2017, 2020)

In [28]:
city_monthly_avg_list = [arco_era5[['2m_temperature', 'total_precipitation', 'total_cloud_cover']].sel(
    longitude=target_lon, 
    latitude=target_lat, method="ffill").sel(
    time=slice(f'{year}-01-01', f'{year+1}-01-01')).resample(time="M").mean() for year in years]

In [13]:
from dask.distributed import Client, performance_report
client = Client()  # Connect to distributed cluster and override default
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 23.94 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51730,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 23.94 GiB
Comm: tcp://127.0.0.1:51748,Total threads: 2
Dashboard: http://127.0.0.1:51753/status,Memory: 5.99 GiB
Nanny: tcp://127.0.0.1:51733,


In [29]:
for item in city_monthly_avg_list[1:]:
    year = int(item["time.year"][0])
    print(year)
    with performance_report(filename=f'./dask_reports/city_monthly_avg_{year}.html'):
        df = item.to_dataframe()
        df.to_csv(f'./era5_monthly/city_monthly_avg_{year}.csv')

2018
2019


In [ ]:
client.close()